**CNN**



In this part, we chose to implement a CNN on the Snacks images.

In [2]:
import pandas as pd
import numpy as np
from PIL import Image
import warnings
warnings.filterwarnings('ignore')
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai import *
from fastai.vision import *
from tensorflow.keras import Sequential,losses
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from skimage.transform import resize

In [3]:
food_train=pd.read_csv('data/food_train.csv')
food_test=pd.read_csv('data/food_test.csv')
nutri=pd.read_csv('data/nutrients.csv')
food_nutri=pd.read_csv('data/food_nutrients.csv')

First, let's define some useful functions to help us load the images

In [18]:
def get_photo(number,category):
    mypath='data/foods_final/train'
    img=Image.open(mypath+'/'+category+'/'+str(number)+'.jpg').convert('RGB')
    return img


def get_photo_test(number):
    mypath='data/foods_final/test'
    img=Image.open(mypath+'/'+str(number)+'.jpg').convert('RGB')
    return img


In [19]:
food_train['photo']=np.nan

food_train['photo'] = food_train.apply(
    lambda x: resize(
        np.asarray(get_photo(x['idx'], x['category'])),
        (100, 100)
    ).astype('float32'),
    axis=1
)
np.array(food_train['photo'].head().tolist()).shape


(5, 100, 100, 3)

In [20]:
food_test['photo']=np.nan

food_test['photo'] = food_test.apply(
    lambda x: resize(
        np.asarray(get_photo_test(x['idx'])),
        (100, 100)
    ).astype('float32'),
    axis=1
)
np.array(food_test['photo'].head().tolist()).shape


(5, 100, 100, 3)

In [21]:
X = np.array(food_train['photo'])
y= pd.Categorical(food_train['category'])
train_y=y.codes
test=np.array(food_test['photo'].tolist())


In [ ]:
tra = np.array(food_train['photo'].tolist())
cat = pd.Categorical(food_train['category'])
tra_y = cat.codes
cat.codes
tes = np.array(food_test['photo'].tolist())


In [ ]:
cat_dict = dict(enumerate(cat.categories))

In [27]:
model = Sequential()
model.add(Conv2D(filters=32, input_shape=(100, 100, 3),
                 kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])
callbacks = [EarlyStopping(monitor='val_loss', patience=5)]
history = model.fit(tra, tra_y, batch_size=120, epochs=30,
                    validation_split=0.1, callbacks=callbacks)
pr = model.predict(tes)
pr = pr.argmax(axis=1)
test_predict_cnn = np.array(list(map(lambda x: cat_dict[x], pr)))
np.savetxt('test_predict_cnn.txt', test_predict_cnn, fmt='%s')

Epoch 1/30
239/239 [==============================] - 68s 280ms/step - loss: 1.5403 - accuracy: 0.3844 - val_loss: 1.4277 - val_accuracy: 0.4443
Epoch 2/30
239/239 [==============================] - 67s 281ms/step - loss: 1.3346 - accuracy: 0.4889 - val_loss: 1.3708 - val_accuracy: 0.4584
Epoch 3/30
239/239 [==============================] - 76s 319ms/step - loss: 1.2298 - accuracy: 0.5407 - val_loss: 1.2930 - val_accuracy: 0.5035
Epoch 4/30
239/239 [==============================] - 81s 338ms/step - loss: 1.1415 - accuracy: 0.5804 - val_loss: 1.2183 - val_accuracy: 0.5560
Epoch 5/30
239/239 [==============================] - 77s 322ms/step - loss: 1.0598 - accuracy: 0.6130 - val_loss: 1.2248 - val_accuracy: 0.5570
Epoch 6/30
239/239 [==============================] - 80s 336ms/step - loss: 0.9936 - accuracy: 0.6363 - val_loss: 1.1946 - val_accuracy: 0.5693
Epoch 7/30
239/239 [==============================] - 79s 331ms/step - loss: 0.9393 - accuracy: 0.6590 - val_loss: 1.2381 - val_ac

We've got only o.6 accuracy.

Let's combine the prediction of the CNN, random forest, and LSTM:
if two or more models agree on a category, we'll select that category.
Otherwise, we'll default to the LSTM's prediction.

In [7]:
test_predict_lstm=np.loadtxt('test_predict_lstm.txt',dtype='str')
test_predict_forest=np.loadtxt('test_predict_RF.txt',dtype='str')
test_predict_cnn=np.loadtxt('test_predict_cnn.txt',dtype='str')

In [8]:
combined_pred = np.where(test_predict_cnn!=test_predict_forest,test_predict_lstm,test_predict_cnn)

In [10]:
pd.Series(combined_pred,food_test['idx'],name='pred_cat').to_csv('model02.csv',index_label='idx')